# Inference Model

### Something's missing?? (To be done later)

*After training, a different model architecture is used to make predictions on new input data, using the trained layers such as LSTM and TimeDistributed Dense which were defined globally during training. The code from chat_bot.py (particualrly the code pertaining to training the model) has to be pasted into this part. I skipped this part to make the entire code simpler to understand.*

In [3]:
import numpy as np
import re
import pickle
import os

from keras.preprocessing import sequence
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed
from keras.models import Model

In [ ]:
# For initial filtering
maxlen = 12
maxLen = 20

In [ ]:
# Import the dictionary
with open('dictionary.pkl', 'rb') as f:
    word_to_index = pickle.load(f)

In [ ]:
# Import the reverse dictionary
with open('reverse_dictionary.pkl', 'rb') as f:
    index_to_word = pickle.load(f)

In [ ]:
# Question asked to the chatbot
question = 'Hey! How are you doing?'

In [ ]:
# Pre-processing to make the data into the format required by the model, same as during training
a = question.split()

for pos,i in enumerate(a):
    a[pos] = re.sub('[^a-zA-Z0-9 .,?!]', '', i)
    a[pos]= re.sub(' +', ' ', i)
    a[pos] = re.sub('([\w]+)([,;.?!#&\'\"-]+)([\w]+)?', r'\1 \2 \3', i)
    
    if len(i.split()) > maxlen:
            a[pos] = (' ').join(a[pos].split()[:maxlen])
            if '.' in a[pos]:
                ind = a[pos].index('.')
                a[pos] = a[pos][:ind+1]

            if '?' in a[pos]:
                ind = a[pos].index('?')
                a[pos] = a[pos][:ind+1]

            if '!' in a[pos]:
                ind = a[pos].index('!')
                a[pos] = a[pos][:ind+1]

In [ ]:
question = ' '.join(a).split()

In [ ]:
# Making the question into an array of the corresponding indexes
question = np.array([word_to_index[w] for w in question])

In [ ]:
# Padding sequences
question = sequence.pad_sequences([question], maxlen = 20)

In [ ]:
# Keras model used to train, so that we define the variables (tensors) that ultimately go into the infernce model
input_context = Input(shape = (maxLen, ), dtype = 'int32', name = 'input_context')
input_target = Input(shape = (maxLen, ), dtype = 'int32', name = 'output_context')

In [ ]:
input_ctx_embed = embed_layer(input_context)
input_tar_embed = embed_layer(input_target)

In [ ]:
encoder_lstm, context_h, context_c = LSTM_encoder(input_ctx_embed)
decoder_lstm, h, _ = LSTM_decoder(input_tar_embed, initial_state = [context_h, context_c],)

In [ ]:
output = dense(decoder_lstm)

In [ ]:
# Define the model for the input (question). Returns the final state vectors of the encoder LSTM
context_model = Model(input_context, [context_h, context_c])

In [ ]:
# Defining the inputs for the decoder LSTM
target_h = Input(shape = (300, ))
target_c = Input(shape = (300, ))

## Decoder LSTM

*Takes in the embedding of the initial word passed as input into the decoder model (the 'BOS' tag), along with the final states of the encoder model, to output the corresponding sequences for 'BOS', and the new LSTM states.*

In [ ]:
target, h, c = LSTM_decoder(input_tar_embed, initial_state = [target_h, target_c])
output = dense(target)
target_model = Model([input_target, target_h, target_c], [output, h, c])

In [ ]:
# Pass in the question to the encoder LSTM, to get the final encoder states of the encoder LSTM
question_h, question_c = context_model.predict(question)

*Initialize the answer that will be generated for the 'BOS' input. Since we have used pre-padding for padding sequences, the last token in the 'answer' variable is initialised with the index for 'BOS'.*

In [ ]:
answer = np.zeros((1, maxLen))
answer[0, -1] = word_to_index['BOS']

In [ ]:
# i keeps track of the length of the generated answer. This won't allow the model to genrate sequences with more than 20 words.
i = 1

In [ ]:
# Making a new list to store the words generated at each time step
answer_1 = []

In [ ]:
# Flag to stop the model when 'EOS' tag is generated or when 20 time steps have passed.
flag = 0

## Running the inference model

In [ ]:
while flag != 1:
    # Making the predictions for the given input token and encoder states
    prediction, prediction_h, prediction_c = target_model.predict([answer, question_h, question_c])
    
    # From the generated predictions of shape (num_examples, maxLen, vocab_size), find the token with max probability
    token_arg = np.argmax(prediction[0, -1, :])
    
    # Appending the corresponding word of the index to the answer_1 list
    answer_1.append(index_to_word[token_arg])
    
    # Set flag to 1 if 'EOS' token is generated or 20 time steps have passed
    if token_arg == word_to_index['EOS'] or i > 20:
        flag = 1
    
    # Re-initialise the answer variable, and set the last token to the output of the current time step. This is then passed
    # as input to the next time step, along with the LSTM states of the current time step
    answer = np.zeros((1,maxLen))
    answer[0, -1] = token_arg
    question_h = prediction_h
    question_c = prediction_c
    
    # Increment the count of the loop
    i+=1

In [ ]:
print (' '.join(answer_1))